In [1]:
import torchvision
import torch

In [2]:
BEST_MODEL_PATH = '/home/jim/Documents/model/steering_0621.pth'

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

model.load_state_dict(torch.load(BEST_MODEL_PATH))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
def gstreamer_pipeline(
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=60,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

In [5]:
import cv2
gst = gstreamer_pipeline(320,180,320,180,framerate=20)
cap = cv2.VideoCapture(gstreamer_pipeline(), cv2.CAP_GSTREAMER)

In [6]:
from IPython.display import clear_output, display
import IPython
from PIL import Image
from io import BytesIO
    
# high speed
# https://medium.com/@kostal91/displaying-real-time-webcam-stream-in-ipython-at-relatively-high-framerate-8e67428ac522
def render_by_jpg(arr, fmt='jpeg'):
    f = BytesIO()
    Image.fromarray(arr).save(f, fmt)
    display(IPython.display.Image(data=f.getvalue()))
    clear_output(wait=True)
    
def render(img):
    return render_by_jpg(img)


In [7]:
from IPython.display import display
import ipywidgets
import traitlets

speed_slider = ipywidgets.FloatSlider(min=-100, max=100.0, step=1, description='speed')
diff_slider = ipywidgets.FloatSlider(min=-100, max=100.0, step=1, description='diff')

display(speed_slider, diff_slider)

FloatSlider(value=0.0, description='speed', min=-100.0, step=1.0)

FloatSlider(value=0.0, description='diff', min=-100.0, step=1.0)

In [9]:
try:
    while True:
        success, img = cap.read()
        if success:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            render(img)
            speed, diff = model(preprocess(img)).detach().float().cpu().numpy().flatten() * 100
            
            speed_slider.value = speed
            diff_slider.value = diff

        else:
            print('camera error')
except KeyboardInterrupt:
    print('stop')

stop


In [ ]:
cap.release()